<a href="https://colab.research.google.com/github/forexms78/AI-05-/blob/main/BERT_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 사전 학습을 위한 데이터세트 구성

BERT 모델은 사전학습과 미세조정(Fine-Tuning)이 확실하게 나눠지는 대표적인 모델입니다.  
미세조정은 자연어 Task에 따라 다양하게 진행 될 수 있지만, 사전 학습은 대부분 NSP 학습과 MLM 학습으로 진행이 됩니다.   

해당 예시에서는 [AIHUB](https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&dataSetSn=86) 한국어 감성 대화 말뭉치를 부분적으로 활용하여 질의->응답 으로 NSP와 MLM 사전 학습을 동시 진행해봅니다.

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving train.csv to train.csv


In [ ]:
import torch
from torch import nn
import copy
from torch.utils.data import Dataset, DataLoader, random_split
import sentencepiece as spm
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv(f'./train.csv')
df[['HS01','SS01']]

,HS01,SS01
0,일은 왜 해도 해도 끝이 없을까? 화가 난다.,많이 힘드시겠어요. 주위에 의논할 상대가 있나요?
1,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요?
2,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 ...
3,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,관련 없는 심부름을 모두 하게 되어서 노여우시군요. 어떤 것이 상황을 나아질 수 있...
4,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,무시하는 것 같은 태도에 화가 나셨군요. 상대방의 어떤 행동이 그런 감정을 유발하는...
...,...,...
51623,나이가 먹고 이제 돈도 못 벌어 오니까 어떻게 살아가야 할지 막막해. 능력도 없고.,경제적인 문제 때문에 막막하시군요. 마음이 편치 않으시겠어요.
51624,몸이 많이 약해졌나 봐. 이제 전과 같이 일하지 못할 것 같아 너무 짜증 나.,건강에 대한 어려움 때문에 기분이 좋지 않으시군요. 속상하시겠어요.
51625,이제 어떻게 해야 할지 모르겠어. 남편도 그렇고 노후 준비도 안 되어서 미래가 걱정돼.,노후 준비에 대한 어려움 때문에 걱정이 많으시겠어요.
51626,몇십 년을 함께 살았던 남편과 이혼했어. 그동안의 세월에 배신감을 느끼고 너무 화가 나.,가족과의 문제 때문에 속상하시겠어요.


### NSP 학습을 위한 두문장 토큰

BERT 사전학습에 있어서 먼저 NSP 학습이 가능하게 하기 위해 2개의 문장을 연결하여 적절하게 연결된 문장과 비적절하게 연결된 문장을 이진 분류 할 수 있게 구성해 줍니다.  

이때 데이터세트에서 만들 최종 토큰은 다음과 같습니다.

    시작토큰[CLS] + 문장1 토큰+ 구분토큰[SEP] + 문장2 토큰 + 구분(종료)토큰[SEP]

    예시: [CLS], 오늘, 수업, 뭐지, [SEP], 자연어, 분석, 수업, [SEP]

이렇게 토큰이 BERT 입력되면 [CLS] 토큰은 모든 토큰과의 연관성이 포함되기 때문에 [CLS] 토큰으로 문장의 연결이 적절한지 분류가 가능해집니다.  

<center><img src="https://drive.google.com/uc?export=view&id=1N1r3JPdmIxXK6QxJASuN_ABpWkgfjm0I" width="400"/></center>

예시에서는 `sentencepiece` 토큰화를 사용하므로 기존에 있는 스페셜 토큰인 `bos`를 `CLS`로 `eos`를 `SEP`으로 활용합니다.




In [ ]:
# BPE :바이트 페어 인코딩(Byte Pair Encoding, BPE)
import os, re

# 추가 쓰기모드로 텍스트 파일 열기
with open('train.txt', 'w', encoding='utf-8') as f:
  for text in df['HS01']:
        text = str(text)
        text = re.sub(r'[^\w\s]', '', text)     # 특수문자 제거
        text = re.sub(r'[\n\t]', ' ', text)     # 줄바꿈, 탭 제거
        text = re.sub(r'\s+', ' ', text)        # 연속된 공백 제거
        text= text.strip()                      # 문장 양끝 공백 제거
        try:
            f.write(text+'\n')
        except:
                pass

# 저장 경로 생성
os.makedirs('./bpe', exist_ok=True)

spm.SentencePieceTrainer.train(
    input='train.txt',                      # 텍스트 뭉치 파일
    model_prefix='./bpe/spm_krsent',        # 출력 모델 파일 이름
    vocab_size=2000                         # 토큰 개수
)

spm.SentencePieceTrainer.train(input='train.txt',               # 텍스트 뭉치 파일
                            model_prefix='./bpe/spm_krsent',    # 출력 모델 파일 이름
                            vocab_size=4000,                    # 토큰 개수
                            bos_id=1,
                            eos_id=2,
                            unk_id=3,
                            pad_id=0
                            )

In [ ]:
class SPDataSet(Dataset):
    def __init__(self, sp, max_len):
        self.max_len = max_len
        self.df = pd.read_csv(f'./train.csv')
        self.sp = sp

        # 두개의 문장을 담을 리스트
        self.pairs = []

        # 적절하게 연결되 문장은 라벨 1로 설정
        for _, item in df.iterrows():
            sent1 = item['HS01']
            sent2 = item['SS01']
            self.pairs.append((sent1, sent2, 1))

        # 비적절한 연결을 만들기 위해 랜덤한 인덱스 생성
        n_lines = len(df)
        rand_indices = np.random.randint(0, n_lines, size=(n_lines-1,))

        # 랜덤하게 연결된 문장은 라벨을 0으로 설정
        for i, rand_idx in enumerate(rand_indices):
            if i == rand_idx:
                continue
            sent1 = df.iloc[i]['HS01']
            sent2 = df.iloc[rand_idx]['SS01']
            self.pairs.append((sent1, sent2, 0))

        # 앞뒤 10개 문장쌍
        print(self.pairs[:10])
        print(self.pairs[-10:])

    # 제로 패딩으 위한 함수
    def zero_pad(self, tok):
        if len(tok) >= self.max_len:
            return tok[:self.max_len]
        else:
            padding = np.zeros(self.max_len)
            padding[:len(tok)] = tok
            return padding

    def __getitem__(self, i):
        sent1, sent2, label = self.pairs[i]
        sent1 = self.sp.encode_as_ids(sent1)
        sent2 = self.sp.encode_as_ids(sent2)

        # 두개의 문장을 bos, eos 토큰과 함께 연결
        inp = [self.sp.bos_id()] + sent1 + [self.sp.eos_id()] + sent2 + [self.sp.eos_id()]

        # 패딩 생성
        inp = self.zero_pad(inp)

        # 패딩 마스크 생성
        mask = torch.eq(torch.Tensor(inp), 0)   # inp 텐서 안에 값이 0인지 비교해서 True/False로 이루어진 텐서를 만든다

        return torch.Tensor(inp), label, mask

    def __len__(self):
        return (len(self.pairs))

sp = spm.SentencePieceProcessor(model_file=f'./bpe/spm_krsent.model')
dataset = SPDataSet(sp, 60)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

for inp, tar, mask in dataloader:
    print(f'입력 토큰 : {inp.long()}')
    print(f'NSP 라벨 : {tar}')
    print(f'패딩 마스크 : {mask}')
    break

[('일은 왜 해도 해도 끝이 없을까? 화가 난다.', '많이 힘드시겠어요. 주위에 의논할 상대가 있나요?', 1), ('이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.', '급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요?', 1), ('회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스트레스 받아. ', '회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 하면 좋을까요?', 1), ('직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 섭섭해.', '관련 없는 심부름을 모두 하게 되어서 노여우시군요. 어떤 것이 상황을 나아질 수 있게 도움을 줄까요?', 1), ('얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.', '무시하는 것 같은 태도에 화가 나셨군요. 상대방의 어떤 행동이 그런 감정을 유발하는 것일까요?', 1), ('직장에 다니고 있지만 시간만 버리는 거 같아. 진지하게 진로에 대한 고민이 생겨.', '진로에 대해서 고민하고 계시는군요. 어떤 점이 고민인가요?', 1), ('성인인데도 진로를 아직도 못 정했다고 부모님이 노여워하셔. 나도 섭섭해.', '부모님의 노여움에 섭섭하시군요. 이런 상황을 어떻게 해결하면 좋을까요? ', 1), ('퇴사한 지 얼마 안 됐지만 천천히 직장을 구해보려고.', '천천히라도 직장을 구해 보려고 하시는군요. 특별한 이유가 있으신가요?', 1), ('졸업반이라서 취업을 생각해야 하는데 지금 너무 느긋해서 이래도 되나 싶어.', '취업에 대해 걱정이 되는군요.', 1), ('요즘 직장생활이 너무 편하고 좋은 것 같아!', '직장생활이 편하고 좋으시다니 좋아 보여요. 다니고 계신 회사만의 장점이 있나요?', 1)]
[('요즘 들어 부쩍 몸이 안 좋아진 거 같아. 아직 애들도 다 안 컸는데 걱정이 돼.', '많이 속상해 보이는군요. 무슨 일이 있었나요?',